# 📚 Public Library Program Engagement (FY 2022)

This notebook analyzes how U.S. public libraries engage their communities through **programs** and **attendance**, based on IMLS FY 2022 data.

---

## 📦 1. Load Data

In [2]:
import pandas as pd

# Load raw IMLS AE dataset
df_raw = pd.read_csv("PLS_FY22_AE_pud22i.csv", encoding="cp1252")

# Select relevant columns
df_programs = df_raw[["STABR", "TOTPRO", "TOTATTEN", "POPU_UND"]].copy()

# Drop missing/invalid values
df_programs = df_programs[
    (df_programs["TOTPRO"] >= 0) &
    (df_programs["TOTATTEN"] >= 0) &
    (df_programs["POPU_UND"] > 0)
]

# Group by state
state_programs = df_programs.groupby("STABR", as_index=False).agg({
    "TOTPRO": "sum",
    "TOTATTEN": "sum",
    "POPU_UND": "sum"
})

# Calculate metrics per 1,000 people
state_programs["Programs per 1k"] = state_programs["TOTPRO"] / (state_programs["POPU_UND"] / 1000)
state_programs["Attendance per 1k"] = state_programs["TOTATTEN"] / (state_programs["POPU_UND"] / 1000)

state_programs.head()

,STABR,TOTPRO,TOTATTEN,POPU_UND,Programs per 1k,Attendance per 1k
0,AK,7827,228907,638242,12.263373,358.652361
1,AL,41146,916800,4898245,8.400151,187.169078
2,AR,36190,774317,2646872,13.672743,292.540402
3,AS,206,3103,51269,4.018023,60.523903
4,AZ,41847,686312,6999792,5.978320,98.047485


---

## 🗺 2. Choropleth: Attendance per 1,000 People by State

In [4]:
import plotly.express as px

fig = px.choropleth(
    state_programs,
    locations='STABR',
    locationmode='USA-states',
    color='Attendance per 1k',
    color_continuous_scale='YlGnBu',
    scope='usa',
    labels={'Attendance per 1k': 'Attendance / 1k'},
    title='📍 Library Program Attendance per 1,000 People by State (FY 2022)'
)
fig.show()

---

## 📈 3. Scatter Plot: Programs per 1k vs. Attendance per 1k

In [6]:
fig = px.scatter(
    state_programs,
    x='Programs per 1k',
    y='Attendance per 1k',
    text='STABR',
    size='POPU_UND',
    title='📊 Program Output vs. Engagement (per 1,000 residents)',
    labels={'Programs per 1k': 'Programs / 1k', 'Attendance per 1k': 'Attendance / 1k'}
)
fig.update_traces(textposition='top center')
fig.show()

---

## 💾 4. Export for Dashboard Use

In [8]:
state_programs.to_csv("PLS_FY22_AE_programs_by_state.csv", index=False)

---

## 🧾 Source

Data from the [Institute of Museum and Library Services (IMLS)](https://imls.gov/research-evaluation/data-collection/public-libraries-survey).